In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('data\\raw\\Airline_Delay_Cause.csv')
df.sample(5)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
22705,2022,6,AA,American Airlines Inc.,TPA,"Tampa, FL: Tampa International",993.0,335.0,117.94,21.81,...,0.51,113.10,68.0,5.0,22458.0,7658.0,1559.0,4023.0,18.0,9200.0
170235,2013,9,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",55.0,8.0,2.53,0.00,...,0.00,2.15,1.0,0.0,348.0,204.0,0.0,83.0,0.0,61.0
1507,2023,8,YX,Republic Airline,ECP,"Panama City, FL: Northwest Florida Beaches Int...",20.0,3.0,0.00,0.00,...,0.00,2.00,0.0,0.0,136.0,0.0,0.0,24.0,0.0,112.0
68513,2020,2,EV,ExpressJet Airlines LLC,LFT,"Lafayette, LA: Lafayette Regional Paul Fournet...",115.0,13.0,2.47,0.93,...,0.00,1.32,0.0,1.0,500.0,71.0,125.0,266.0,0.0,38.0
170946,2013,8,F9,Frontier Airlines Inc.,GSO,"Greensboro/High Point, NC: Piedmont Triad Inte...",13.0,3.0,1.00,0.00,...,0.00,2.00,0.0,0.0,135.0,80.0,0.0,0.0,0.0,55.0


In [5]:
df.year.unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013])

In [6]:
df.groupby('year').size()

year
2013     6733
2014    13980
2015    13528
2016    12217
2017    12518
2018    20213
2019    20946
2020    18859
2021    19954
2022    20345
2023    12373
dtype: int64